In [1]:
################################################
# 加载数据和特征工程
################################################

%run load_data.ipynb
%run build_features.ipynb

In [ ]:
################################################
# 准备数据格式
################################################
import lightgbm as lgb

features = combined_data.iloc[:n_original, :-1]
targets = combined_data.iloc[:n_original, -1]

all_data = lgb.Dataset(features, label=targets)


In [3]:
################################################
# 加载最佳超参数
################################################
import json
import chardet

# Detect the file encoding
with open('../data/best_hyper_parameters.json', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

# Read the JSON content from the file using the detected encoding
with open('../data/best_hyper_parameters.json', 'r', encoding=encoding) as file:
    data = json.load(file)

# Extract the best_params and best_estimator
best_params = data["best_params"]
best_estimator = data["best_estimator"]

print(best_params)
print(best_estimator["n_estimators"])

{'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'None', 'bagging_fraction': 0.8, 'bagging_freq': 5, 'feature_fraction': 0.9, 'learning_rate': 0.02, 'max_depth': 9, 'num_leaves': 127, 'min_child_samples': 20}
8700


In [4]:
################################################
# 训练模型
################################################

best_gbm = lgb.train(
    best_params,
    all_data,
    num_boost_round=best_estimator["n_estimators"]
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1236
[LightGBM] [Info] Number of data points in the train set: 25782, number of used features: 204
[LightGBM] [Info] Start training from score 54.437405
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [5]:
eval_data = combined_data.iloc[n_original:, :-1]
y_pred = best_gbm.predict(eval_data)
y_pred[y_pred>100] = 100
y_pred[y_pred<0] = 0

In [7]:
from datetime import datetime

df_submit["mRNA_remaining_pct"] = y_pred

timestamp = datetime.now().strftime("%Y%m%d")
filename = f"../submit/submission_{timestamp}.csv"
print(f"File saved as {filename}")

df_submit.to_csv(filename, index=False)

File saved as ../submit/submission_20240821.csv
